Convert Global Wheat Detection DS the train.csv to per image labelme json files

In [1]:
import pandas as pd
from IPython.display import Image
import json
import os
from tqdm import tqdm

In [3]:
d = pd.read_csv('./dataset/global-wheat-detection/train.csv')
d['image_id'].unique()[0]
for image_id in tqdm(list(d['image_id'].unique())):
    shapes = []
    for _, row in d[d['image_id']==image_id].iterrows():
        x, y, w, h = json.loads(row['bbox'])
        shapes.append({
          "label": "wheat",
          "points": [[x, y], [x+w, y+h]],
          "group_id": None,
          "shape_type": "rectangle",
          "flags": {}
        })
    img_filename = "{}.jpg".format(image_id)
    labelme = json.dumps({
      "version": "4.0.0",
      "flags": {},
      "shapes": shapes,
      "imagePath": img_filename,
      "imageData": None,
      "imageHeight": 1024,
      "imageWidth": 1024
    }, indent=2)
    annot_filename = "labelme/{}.json".format(image_id)
    os.makedirs(os.path.dirname(annot_filename), exist_ok=True)
    with open(annot_filename, "w") as f:
        f.write(labelme)

100%|██████████| 3373/3373 [00:42<00:00, 80.07it/s]


In [4]:
import os
import shutil

# Caminhos das pastas
dataset_path = r'C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\dataset\global-wheat-detection'
labels_path = r'C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\labelme'

# Listar todos os arquivos .json na pasta de labels
label_files = [f for f in os.listdir(labels_path) if f.endswith('.json')]

# Mover cada arquivo .json para a pasta correspondente
for label_file in label_files:
    # Nome do arquivo sem extensão
    file_name = os.path.splitext(label_file)[0]
    
    # Caminho completo do arquivo de label
    src_path = os.path.join(labels_path, label_file)
    
    # Verificar se o arquivo de imagem correspondente está na pasta train ou test
    if os.path.exists(os.path.join(dataset_path, 'train', f'{file_name}.jpg')):
        dest_path = os.path.join(dataset_path, 'train', label_file)
    elif os.path.exists(os.path.join(dataset_path, 'test', f'{file_name}.jpg')):
        dest_path = os.path.join(dataset_path, 'test', label_file)
    else:
        print(f'Imagem correspondente para {label_file} não encontrada em train ou test.')
        continue
    
    # Mover o arquivo de label para a pasta correspondente
    shutil.move(src_path, dest_path)
    print(f'Movido {label_file} para {dest_path}')

Movido 00333207f.json para C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\dataset\global-wheat-detection\train\00333207f.json
Movido 005b0d8bb.json para C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\dataset\global-wheat-detection\train\005b0d8bb.json
Movido 006a994f7.json para C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\dataset\global-wheat-detection\train\006a994f7.json
Movido 00764ad5d.json para C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\dataset\global-wheat-detection\train\00764ad5d.json
Movido 00b5fefed.json para C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\dataset\global-wheat-detection\train\00b5fefed.json
Movido 00b70a919.json para C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentat